# 1. Implement A* search on a maze (given as a 2D matrix where 1=wall, 0=path, A=start, B=goal).

#  - Use Manhattan distance as heuristic. 

In [80]:
def manhattan(a, b):   
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

- Visualize the path found. 


In [81]:
import heapq
def astar(maze, start, goal):
    rows, cols = maze.shape
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}
    f_score = {start: manhattan(start, goal)}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
          
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  

      
        neighbors = [(0,1), (1,0), (0,-1), (-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)

            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if maze[neighbor] == 1: 
                    continue

                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + manhattan(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None 


maze= [
    ['A',0, 0, 1, 0, 0, 0],
    [1,  1, 0, 1, 0, 1, 0],
    [1,  0, 0, 0, 0, 1, 0],
    [1,  0, 1, 1, 1, 1, 0],
    [1,  0, 0, 0, 0, 0, 'B']
]


import numpy as np
maze_np = np.zeros((len(maze), len(maze[0])), dtype=int)
start, goal = None, None

for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j] == 'A':
            start = (i, j)
            maze_np[i, j] = 0
        elif maze[i][j] == 'B':
            goal = (i, j)
            maze_np[i, j] = 0
        else:
            maze_np[i, j] = maze[i][j]



path = astar(maze_np, start, goal)
print("Path:", path)
print("Path length:", len(path))



Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6)]
Path length: 13


Case 1: Multiply Manhattan distance by 1.5 (check if it is admissible).


In [82]:
def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def manhattan_scaled(a, b):
    return 1.5 * (abs(a[0] - b[0]) + abs(a[1] - b[1]))


def astar(maze, start, goal, heuristic):
    rows, cols = maze.shape
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  

        neighbors = [(0,1), (1,0), (0,-1), (-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)

            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if maze[neighbor] == 1: 
                    continue

                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None


maze = [
    ['A', 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 'B']
]

maze_np = np.zeros((len(maze), len(maze[0])), dtype=int)
start, goal = None, None

for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j] == 'A':
            start = (i, j)
            maze_np[i, j] = 0
        elif maze[i][j] == 'B':
            goal = (i, j)
            maze_np[i, j] = 0
        else:
            maze_np[i, j] = maze[i][j]


path = astar(maze_np, start, goal, heuristic=manhattan_scaled)

print("=== Case 1: 1.5 × Manhattan Heuristic ===")
print("Path:", path)
print("Path length:", len(path) if path else None)

print("\nAdmissibility check: NOT admissible, because 1.5× Manhattan can overestimate true path cost.")


=== Case 1: 1.5 × Manhattan Heuristic ===
Path: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Path length: 9

Admissibility check: NOT admissible, because 1.5× Manhattan can overestimate true path cost.


Case 2: Define heuristic so that for one edge it violates consistency.

In [83]:
def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def inconsistent(a, b):
    if a == (0, 1):
        return 10  
    return abs(a[0] - b[0]) + abs(a[1] - b[1])



def astar(maze, start, goal, heuristic):
    rows, cols = maze.shape
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        neighbors = [(0,1), (1,0), (0,-1), (-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)

            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if maze[neighbor] == 1: 
                    continue

                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None 



maze= [
    ['A', 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 'B']
]

import numpy as np
maze_np = np.zeros((len(maze), len(maze[0])), dtype=int)
start, goal = None, None

for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j] == 'A':
            start = (i, j)
            maze_np[i, j] = 0
        elif maze[i][j] == 'B':
            goal = (i, j)
            maze_np[i, j] = 0
        else:
            maze_np[i, j] = maze[i][j]


print("\nCase 2: Inconsistent heuristic at (0,1)")
path2 = astar(maze_np, start, goal, inconsistent)
print("Path:", path2)
print("Path length:", len(path2))



Case 2: Inconsistent heuristic at (0,1)
Path: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (3, 2), (3, 3), (3, 4), (4, 4)]
Path length: 11


3. For each case, run A* and record: - 
Path found - 
Cost of path - 
Whether the path is optima

In [85]:
from collections import deque

def bfs_shortest_path(maze, start, goal):
    rows, cols = maze.shape
    q = deque([(start, [start])])
    visited = {start}
    while q:
        (x, y), path = q.popleft()
        if (x, y) == goal:
            return path
        for dx, dy in [(0,1),(1,0),(0,-1),(-1,0)]:
            nx, ny = x+dx, y+dy
            if 0<=nx<rows and 0<=ny<cols and maze[nx, ny]==0 and (nx, ny) not in visited:
                visited.add((nx, ny))
                q.append(((nx, ny), path+[(nx, ny)]))
    return None

maze= [
    ['A', 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 'B']
]

maze_np = np.zeros((len(maze), len(maze[0])), dtype=int)
start, goal = None, None

for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j]=='A':
            start=(i,j)
            maze_np[i,j]=0
        elif maze[i][j]=='B':
            goal=(i,j)
            maze_np[i,j]=0
        else:
            maze_np[i,j]=maze[i][j]

true_path = bfs_shortest_path(maze_np, start, goal)
true_cost = len(true_path)-1

path1 = astar(maze_np, start, goal, manhattan_scaled)
cost1 = len(path1)-1 if path1 else None
optimal1 = cost1==true_cost

print("=== Case 1: 1.5× Manhattan Heuristic ===")
print("Path found:", path1)
print("Cost of path:", cost1)
print("Is path optimal?", optimal1)
print("Admissibility check: NOT admissible (may overestimate)\n")


path2 = astar(maze_np, start, goal, inconsistent)
cost2 = len(path2)-1 if path2 else None
optimal2 = cost2==true_cost

print("=== Case 2: Inconsistent heuristic at (0,1) ===")
print("Path found:", path2)
print("Cost of path:", cost2)
print("Is path optimal?", optimal2)


=== Case 1: 1.5× Manhattan Heuristic ===
Path found: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Cost of path: 8
Is path optimal? True
Admissibility check: NOT admissible (may overestimate)

=== Case 2: Inconsistent heuristic at (0,1) ===
Path found: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (3, 2), (3, 3), (3, 4), (4, 4)]
Cost of path: 10
Is path optimal? False
